In [1]:
!pip install transformers
!pip install PersianStemmer
!pip install https://github.com/htaghizadeh/PersianStemmer-Python/archive/master.zip --upgrade
!pip install hazm
!pip install demoji


     |████████████████████████████████| 2.6 MB 29.1 MB/s 
     |████████████████████████████████| 3.3 MB 38.9 MB/s 
     |████████████████████████████████| 636 kB 29.8 MB/s 
     |████████████████████████████████| 895 kB 34.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 810 kB 25.6 MB/s 
  Created wheel for PersianStemmer: filename=PersianStemmer-1.0.0-py3-none-any.whl size=810281 sha256=d2c1242d832535d7a542db0c77c4c349425288a9b2ac589f1dfb12ba10c29b25
  Stored in directory: /root/.cache/pip/wheels/d2/f7/96/55f0070725e6ce1191aa9a931e962fc0f551b68ed919fb7f8e
  Created wheel for patricia-trie: filename=patricia_trie-10-py3-none-any.whl size=7768 sha256=3d4d9299e13aa8ac8057be08a91c59f52ee01940827be52b7f2e465a0313be87
  Stored in directory: /root/.cache/pip/wheels/32/89/b4/46456cda461ffee3f6ef09a68bccb5d175b65ff2b922b5b5b8
Successfully built P

In [2]:
from google.colab import drive
import sys
drive.mount('/content/drive')
sys.path.insert(0, '/content/drive/My Drive/modules')

Mounted at /content/drive


In [3]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report, accuracy_score, f1_score, recall_score,precision_score, make_scorer
from sklearn.model_selection import StratifiedKFold, GridSearchCV
import preprocess
from statistics import mean
import demoji
demoji.download_codes()

... OK (Got response in 0.13 seconds)
Writing emoji data to /root/.demoji/codes.json ...
... OK


In [4]:
def run_rf():
    RF_model.fit(x_train, y_train)
    prediction_RF = RF_model.predict(x_test)
    f_score = round(((f1_score(y_test, prediction_RF)) * 100), 2)
    accuracy = round((accuracy_score(y_test, prediction_RF) * 100), 2)
    precision = round((precision_score(y_test, prediction_RF) * 100), 2)
    recall = round((recall_score(y_test, prediction_RF) * 100), 2)
    return f_score, accuracy, precision, recall

In [5]:
stop_words = 'custom'
stemming = True
dataset = preprocess.auto_preprocess(pd.read_csv('/content/drive/MyDrive/data/initial_dataset.csv'), stop_words_type=stop_words, stemming=stemming)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

custom stop words selected



there are 5 outlier in data
current dataframe length before remove 5 outlier sentence is: 2000
current dataframe length after remove 5 outlier sentence is: 1995
auto preprocess completed done on dataframe!
Enjoy!


In [7]:
vectorizer = CountVectorizer(ngram_range=(1,1), min_df=4, max_df=0.8)
x = vectorizer.fit_transform(dataset['sentence'])
y = (dataset['label'].to_numpy())
RF_model = RandomForestClassifier(random_state=42) 
params = {'class_weight': {0: 1, 1: 5}, 'criterion': 'entropy', 'max_depth': 7, 'max_features': 'auto', 'n_estimators': 200}
RF_model.set_params(**params)

f1_list = []
accuracy_list = []
precision_list = []
recall_list = []

for train_index, test_index in skf.split(dataset['sentence'], dataset['label']):
    x_train, x_test = x[train_index], x[test_index]
    y_train, y_test = y[train_index], y[test_index]
    f1, accuracy, precision, recall = run_rf()
    f1_list.append(round(f1, 2))
    accuracy_list.append(round(accuracy, 2))
    precision_list.append(round(precision, 2))
    recall_list.append(round(recall,2))

print (f"f1 is: {mean(f1_list)}\naccuracy is: {mean(accuracy_list)}\nprecision is: {mean(precision_list)}\nrecall is: {mean(recall_list)}")      

f1 is: 74.876
accuracy is: 91.88000000000001
precision is: 79.668
recall is: 70.884
